## Homework 1 - Part 2 
## Recursive Best First Search Method to solve the Romanian Route Finding problem of Combinatorial Optimization

#### Name : Sowmya Sivakumar
#### Andrew ID : sowmyasi@andrew.cmu.edu

### Objective : To find the shortest route between Oradea and Hirsova using the RBFS algorithm


In [56]:
import math
import csv
import pandas as pd
import numpy as np
import time 
infinity = float('inf')
#start time 
start_time = time.time()


In [57]:
class Graph:
    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed

    def get(self, a, b=None):
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)


In [58]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.f = 0  # extra variable to represent total cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def expand(self, problem):
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):  # to make node object of each child
        next_state = problem.result(self.state, action)
        new_cost = problem.path_cost(self.path_cost, self.state, action,
                                     next_state)
        next_node = Node(next_state, self, action, new_cost)
        return next_node

    def solution(self):  # extracts the path of solution
        return [node.state for node in self.path()]

    def path(
            self):  # extracts the path of any node starting from current to source
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(
            path_back))  # order changed to show from source to current


In [59]:
def is_in(elt, seq): 
    return any(x is elt for x in seq)

In [60]:
class GraphProblem(object):
    def __init__(self, initial, goal, graph):
        self.graph = graph
        self.initial = initial
        self.goal = goal

    def actions(self, A):
        return list(self.graph.get(A).keys())

    def result(self, state, action):
        return action

    def path_cost(self, cost_so_far, A, action, B):
        return cost_so_far + (self.graph.get(A, B) or infinity)

    def goal_test(self, state):
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def h(self, node):
        locs = getattr(self.graph, 'locations', None)
        if locs:
            if type(node) is str:
                return int(get_distance(locs[node], locs[self.goal]))
            return int(
                get_distance(locs[node.state], locs[self.goal]))  
        else:
            return infinity


In [61]:
def mymax(childf, nodef, child, node):
    if childf >= nodef:
        print("node=", node.state, ", child=", child.state,
              ", node f=", nodef, " childf = ", childf, " assigning child's f")
        return childf
    else:
        print("node=", node.state, ", child=", child.state,
              ", node f=", nodef, " childf = ", childf,
              " assigning node's  f <----")
        return nodef

In [62]:
def get_distance(a, b):
    """helper function that calculates the Euclidean distance between
    two (x, y) points."""
    xA, yA = a
    xB, yB = b
    return math.hypot((xA - xB), (yA - yB))

In [63]:
def lowest_fvalue_node(nodelist):
    
#     if len(nodelist) == 0:
#         return 0
    
    min_fval = nodelist[0].f
    min_fval_node_index = 0
    for n in range(1, len(nodelist)):
        if nodelist[n].f < min_fval:
            min_fval_node_index = n
            min_fval = nodelist[n].f
    return nodelist[min_fval_node_index]


def second_lowest_fvalue(nodelist,lowest_f): 
 
    secondmin_fval = infinity
    for n in range(0,len(nodelist)):
        if nodelist[n].f > lowest_f.f and nodelist[n].f < secondmin_fval :
            secondmin_fval = nodelist[n].f
    return secondmin_fval


In [64]:
def RecursiveBFS(problem):
    startnode = Node(problem.initial)
    startnode.f = problem.h(problem.initial)
    return RBFS(problem, startnode, infinity)


In [65]:

def RBFS(problem, node, f_limit):
    print("\nIn RBFS Function with node ", node.state,
          " with node's f value = ", node.f, " and f-limit = ", f_limit)
    if problem.goal_test(node.state):
        return [node, None]
    successors = []
    for child in node.expand(problem):
        gval = child.path_cost
        hval = problem.h(child)
        child.f = mymax(gval + hval, node.f, child, node)
        successors.append(child)
     
    # *********************************************************************
        print("\n Got following successors for  ", node.state, ":", successors)
        if len(successors) == 0:
            return [None, infinity]
    while True:
        best = lowest_fvalue_node(successors)
        
        if best.f > f_limit:
            return [None, best.f]
        
        if len(successors) != 1:
            alternative = second_lowest_fvalue(successors, best)
        
        x = RBFS(problem, best, min(f_limit, alternative))
        # *********************************************************************
        result = x[0]
        print("updating f value of best node ", best.state, " from ",
              best.f, " to ", x[1])
        best.f = x[1]
        if result != None:
            return [result, None]


In [66]:
# read data from the source files into suggested data formats 
if __name__ == '__main__':

    romania_map = {}
    cityDistances = pd.read_csv("p2-distances.csv", names= ['from','to','distance'])

    for city in cityDistances['from'].unique():

        cityData = cityDistances[cityDistances['from']==city]
        cityDict = dict(zip(cityData['to'],cityData['distance']))
        romania_map[city] = cityDict
    
    romania_graph = Graph(graph_dict=romania_map, directed=False)
    romaniaGraphLocs = {}

    cityLocations = pd.read_csv("p2-locations.csv.")
    for index, row in cityLocations.iterrows():
        romaniaGraphLocs[row['location']] = (row['x'],row['y'])
        
    romania_graph.locations = romaniaGraphLocs
    print(romania_graph.locations)
    
    print("\n\nSolving for Oradea to Hirsova....")
    romania_problem = GraphProblem('Oradea', 'Hirsova', romania_graph)
    
    resultnode = RecursiveBFS(romania_problem)
    end_time = time.time()
    if (resultnode[0] != None):
        print("Path taken :", resultnode[0].path())
        print("Path Cost :", resultnode[0].path_cost)
        print("Total time taken :", end_time - start_time)



{'Arad': (91, 492), 'Bucharest': (400, 327), 'Craiova': (253, 288), 'Drobeta': (165, 299), 'Eforie': (562, 293), 'Fagaras': (305, 449), 'Giurgiu': (375, 270), 'Hirsova': (534, 350), 'Iasi': (473, 506), 'Lugoj': (165, 379), 'Mehadia': (168, 339), 'Neamt': (406, 537), 'Oradea': (131, 571), 'Pitesti': (320, 368), 'Rimnicu': (233, 410), 'Sibiu': (207, 457), 'Timisoara': (94, 410), 'Urziceni': (456, 350), 'Vaslui': (509, 444), 'Zerind': (108, 531)}


Solving for Oradea to Hirsova....

In RBFS Function with node  Oradea  with node's f value =  459  and f-limit =  inf
node= Oradea , child= Zerind , node f= 459  childf =  533  assigning child's f

 Got following successors for   Oradea : [<Node Zerind>]
node= Oradea , child= Sibiu , node f= 459  childf =  495  assigning child's f

 Got following successors for   Oradea : [<Node Zerind>, <Node Sibiu>]

In RBFS Function with node  Sibiu  with node's f value =  495  and f-limit =  533
node= Sibiu , child= Arad , node f= 495  childf =  756  assign


### Results :
#### The shortest path between Oradea to Hirsova using the Recursive Best First Search Method is as follows : 
#### Oradea -> Sibiu -> Rimnicu -> Pitesti -> Bucharest -> Urziceni -> Hirsova
#### Path distance : 612 kilometers 
#### Time taken for computation : 0.37 seconds
#### The recursive best first search solves this problem by using the f(n) which is the distance from the beginning to the current node + a heuristic distance of the current node to the goal(which is calculated using the Euclidean distance). This problem stores only the alternative best path at a given time and does not store all the list of nodes that have been visited. Hence, it works exceedingly fast. It does however, have the problem of going in loops since it can visit the same nodes more than once. 